<a href="https://colab.research.google.com/github/SidU/LLMs-from-scratch/blob/main/BytePairEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Byte Pair Encoding (BPE)

### Introduction
Byte Pair Encoding (BPE) is a popular tokenization algorithm used in Natural Language Processing (NLP). It helps break down text into subword units, making it easier for models to handle rare words and unknown tokens efficiently.

### Objectives
- Understand how BPE works step by step.
- Implement BPE using Python.
- Visualize the process with real examples.

This notebook will guide you through the algorithm and its application.


In [18]:
# Import necessary libraries
import collections
import re

### Step 1: Initial Dataset
To demonstrate BPE, we'll use a small dataset of words. These words will be split into characters for initial processing.


In [19]:
# Sample dataset
corpus = [
    "low",
    "lowest",
    "new",
    "wider"
]

# Display the dataset
print("Initial corpus:", corpus)

Initial corpus: ['low', 'lowest', 'new', 'wider']


### Step 2: Tokenization at Character Level
We start by splitting each word in the dataset into characters. We'll also add a special marker `</w>` to indicate the end of a word.


In [20]:
# Function to tokenize each word into characters
def tokenize_corpus(corpus):
    return [" ".join(word) + " </w>" for word in corpus]  # Add </w> to mark end of word

# Tokenize the corpus
tokenized_corpus = tokenize_corpus(corpus)

# Display tokenized words
print("Tokenized corpus:", tokenized_corpus)

Tokenized corpus: ['l o w </w>', 'l o w e s t </w>', 'n e w </w>', 'w i d e r </w>']


### Step 3: Count Pair Frequencies
Next, we'll count the frequency of consecutive pairs of tokens (characters or subwords) across the entire dataset.

In [21]:
# Function to count consecutive token pair frequencies
def get_pair_frequencies(corpus):
    pairs = collections.defaultdict(int)
    for word in corpus:
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i + 1])] += 1
    return pairs

# Count pair frequencies
pair_frequencies = get_pair_frequencies(tokenized_corpus)

# Display pair frequencies
print("\n".join(f"{pair}: {frequency}" for pair, frequency in pair_frequencies.items()))


('l', 'o'): 2
('o', 'w'): 2
('w', '</w>'): 2
('w', 'e'): 1
('e', 's'): 1
('s', 't'): 1
('t', '</w>'): 1
('n', 'e'): 1
('e', 'w'): 1
('w', 'i'): 1
('i', 'd'): 1
('d', 'e'): 1
('e', 'r'): 1
('r', '</w>'): 1


### Step 4: Merge the Most Frequent Pair
The most frequent pair of tokens will be merged into a single token. This process reduces the number of tokens and captures frequent patterns.

In [22]:
# Function to merge the most frequent pair in the corpus
def merge_pair(pair, corpus):
    merged_corpus = []
    bigram = re.escape(" ".join(pair))
    pattern = re.compile(rf"(?<!\S){bigram}(?!\S)")
    for word in corpus:
        merged_word = pattern.sub("".join(pair), word)
        merged_corpus.append(merged_word)
    return merged_corpus

# Identify the most frequent pair
most_frequent_pair = max(pair_frequencies, key=pair_frequencies.get)

# Merge the most frequent pair
tokenized_corpus = merge_pair(most_frequent_pair, tokenized_corpus)

# Display updated corpus after merging
print("Updated corpus after merging:", tokenized_corpus)

Updated corpus after merging: ['lo w </w>', 'lo w e s t </w>', 'n e w </w>', 'w i d e r </w>']


### Step 5: Iterative Merging
We repeat the process of counting pair frequencies and merging the most frequent pair until the desired vocabulary size is reached.

In [23]:
# Perform BPE for a fixed number of iterations
def perform_bpe(corpus, num_merges):

    for _ in range(num_merges):

        # Step 1: Count pair frequencies
        pairs = get_pair_frequencies(corpus)
        if not pairs:
            break

        # Step 2: Find the most frequent pair
        most_frequent_pair = max(pairs, key=pairs.get)

        # Step 3: Merge the most frequent pair
        corpus = merge_pair(most_frequent_pair, corpus)

        # Display progress
        print(f"Most frequent pair: {most_frequent_pair}")
        print("Updated corpus:", corpus)

    return corpus

# Run BPE with 10 merges
final_corpus = perform_bpe(tokenized_corpus, num_merges=10)

Most frequent pair: ('lo', 'w')
Updated corpus: ['low </w>', 'low e s t </w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('low', '</w>')
Updated corpus: ['low</w>', 'low e s t </w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('low', 'e')
Updated corpus: ['low</w>', 'lowe s t </w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('lowe', 's')
Updated corpus: ['low</w>', 'lowes t </w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('lowes', 't')
Updated corpus: ['low</w>', 'lowest </w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('lowest', '</w>')
Updated corpus: ['low</w>', 'lowest</w>', 'n e w </w>', 'w i d e r </w>']
Most frequent pair: ('n', 'e')
Updated corpus: ['low</w>', 'lowest</w>', 'ne w </w>', 'w i d e r </w>']
Most frequent pair: ('ne', 'w')
Updated corpus: ['low</w>', 'lowest</w>', 'new </w>', 'w i d e r </w>']
Most frequent pair: ('new', '</w>')
Updated corpus: ['low</w>', 'lowest</w>', 'new</w>', 'w i d e r </w>']
Most frequent pai

### Step 6: Visualizing the Final Vocabulary
After applying BPE, the corpus will consist of subwords that are frequently occurring patterns. These subwords form the final vocabulary.

In [ ]:
# Extract final vocabulary
def extract_vocabulary(corpus):
    vocab = set()
    for word in corpus:
        vocab.update(word.split())
    return vocab

# Display the final vocabulary
final_vocabulary = extract_vocabulary(final_corpus)
print("Final Vocabulary:", final_vocabulary)

Final Vocabulary: {'r', 'd', 'lowest</w>', 'low</w>', 'wi', 'e', 'new</w>', '</w>'}


### Conclusion
This notebook demonstrates how Byte Pair Encoding (BPE) works step by step. By iteratively merging the most frequent token pairs, BPE generates a compact vocabulary of subwords, which is widely used in NLP for efficient tokenization.

### **Appendix - 1: The End-of-Word Marker (`</w>`)**

In Byte Pair Encoding (BPE), the `</w>` marker is added to indicate the end of a word. This ensures that subwords appearing at the end of a word are treated differently from those in the middle of another word. For example:
- The word `low` becomes `l o w </w>` to indicate it's a complete word.
- This prevents confusion between `low` as a standalone word and `low` as part of another word, like `lowest`.

Here’s how to tokenize the corpus with the `</w>` marker:


In [ ]:
# Function to tokenize each word into characters with an end-of-word marker
def tokenize_corpus(corpus):
    return [" ".join(word) + " </w>" for word in corpus]  # Add </w> to mark the end of the word

# Example: Tokenizing with the end-of-word marker
corpus = ["low", "lowest", "new", "wider"]
tokenized_corpus = tokenize_corpus(corpus)

# Display the tokenized corpus with end-of-word markers
print("Tokenized Corpus with </w> marker:", tokenized_corpus)

Tokenized Corpus with </w> marker: ['l o w </w>', 'l o w e s t </w>', 'n e w </w>', 'w i d e r </w>']


### **Appendix - 2: Byte Pair Encoding with `tiktoken` Library**

In this section, we'll use the `tiktoken` library to perform BPE tokenization. `tiktoken` is an open-source tokenizer library developed by OpenAI, commonly used with GPT models. It provides efficient implementations of BPE and other tokenization algorithms.

#### **Installation**

First, we need to install the `tiktoken` library:


In [ ]:
# Install tiktoken library
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00


#### Using tiktoken for BPE Tokenization
The tiktoken library comes with pre-trained BPE encoders. For demonstration purposes, we'll use the cl100k_base encoder, which is used by models like gpt-3.5-turbo.

In [ ]:
import tiktoken

# Load the BPE encoder
encoding = tiktoken.get_encoding("cl100k_base")

#### Tokenizing Text with BPE
Let's tokenize some sample text using the BPE encoder.

In [ ]:
# Sample text
text = "This is an example of Byte Pair Encoding using tiktoken library."

# Tokenize the text
tokens = encoding.encode(text)

# Display the tokens
print("Tokens:", tokens)

Tokens: [10516, 15821, 502, 22622]


#### Decoding Tokens Back to Text
We can also decode the tokens back to the original text to verify the correctness.

In [ ]:
# Decode the tokens back to text
decoded_text = encoding.decode(tokens)

# Display the decoded text
print("Decoded Text:", decoded_text)


Decoded Text: low lowest new wider


### Inspecting the Tokens and Their Corresponding Subwords
Let's see what subwords each token corresponds to.

In [ ]:
# Display tokens with their corresponding subwords
token_subwords = [encoding.decode([token]) for token in tokens]
print("Token - Subword Mapping:")
for token, subword in zip(tokens, token_subwords):
    print(f"{token}: '{subword}'")


Token - Subword Mapping:
10516: 'low'
15821: ' lowest'
502: ' new'
22622: ' wider'
